# Web Scraping TV Show Data from Various Sources for Data Analysis and Visualization using Python
##### By Wayne Omondi

### Introduction

For time to time, the data we need for a project - personal or professional - may not readily available in a csv file on your drive or on a downloadable format on Kaggle or on Our World In Data. This is where web scraping comes in.<br>
***Web scraping*** is the process of extracting and parsing data from websites. It's a useful technique for creating your own datasets for research and learning. The scraping process involves 'downloading', parsing and processing HTML documents from our target pages.<br> The steps to take will be:

- Picking a website(s) and identifying the information to scrape from the site based on objective(s).
- Using the requests library to 'get' web page(s) locally
- Inspecting the webpage's HTML source and knowing the tags that contains the data needed.
- Using Beautiful Soup to parse (break into components) and extracting relevant information from the html document into a dataframe.
- Cleaning the data.
- Merging the dataframes.
- Exporting the scraped data to our drive.

For this project the target TV Show is **Criminal Minds**, one of my personal favourites. In my opinion, the show did 'fall-off' in the later seasons and I'd like to see if the data speaks to that and the overall data on the show and its perfomances during the seasons it aired for (15 in total).<br>

The data for the TV show will come from IMDB and Wikipedia. IMDB will include a Summary, Ratings and Votes for each episode, while the Wikipedia page will contain the Viewers (in millions) for each episodes: we will create a dataframe from both websites and then merge them into one dataset with all the data we need for analysis and vizualizations.

### 1.0: Libraries/Tools 

In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
!pip install lxml --quiet
!pip install requests --quiet

In [3]:
#get() send a GET request to the specified url
#bs4 lib for pulling data out of HTML/XML files
#pandas for data manipulation

from requests import get 
from bs4 import BeautifulSoup 
import pandas as pd 

### 2.0: Data Collection

#### 2.1: Scraping Data from Wikipedia

In [4]:
#first target website is wikipedia
#list of criminalminds' episodes on wikipedia. 
#the data here is in a table hence read_html() will be a faster and simpler option
wiki_url = 'https://en.wikipedia.org/wiki/List_of_Criminal_Minds_episodes' 

In [5]:
#using read_html() method to get the tabular data for the html doc
wiki_html = pd.read_html(wiki_url)

#view the first two rows of the first table for the html document
wiki_html[0].head(1) 

Season Episodes               Originally aired               Rank Rating
  Season Episodes Episodes.1         First aired    Last aired Rank Rating
0      1       22         22  September 22, 2005  May 10, 2006   27    8.2

In [6]:
#how many tables are in the doc
len(wiki_html) 

20

In [7]:
#iterate through all table elements to view them 
#note data to extract based on their index

#for i, t in enumerate(wiki_html): 
#    print("***********************************") #a separator between each table element
    
    #show the index and table
#    print(i) 
#    print(t)

Based on the table outputs, we want indices 1 to 15 which should cover seasons 1 to 15 of the show.<br> 
While at it we can see that the table with the last season (15) had a different column name than the previous. It is _'U.S. viewers (millions)'_ while the rest are 'US viewers (millions)'

In [8]:
#based on the above output season 15 of criminalminds is index 15
wiki_html[15] 

,No. overall,No. in season,Title,Directed by,Written by,Original air date,Prod. code,U.S. viewers (millions)
0,315,1,"""Under the Skin""",Nelson McCormick,Christopher Barbour,"January 8, 2020",1416,4.82[311]
1,316,2,"""Awakenings""",Alec Smight,Stephanie Sengupta,"January 8, 2020",1417,4.49[311]
2,317,3,"""Spectator Slowing""",Kevin Berlandi,Bruce Zimmerman,"January 15, 2020",1418,4.58[312]
3,318,4,"""Saturday""",Edward Allen Bernero,Stephanie Birkitt & Breen Frazier,"January 22, 2020",1419,4.49[313]
4,319,5,"""Ghost""",Diana Valentine,Bobby Chacon & Jim Clemente,"January 29, 2020",1421,5.88[314]
5,320,6,"""Date Night""",Marcus Stokes,Breen Frazier,"February 5, 2020",1420,4.35[315]
6,321,7,"""Rusty""",Rachel Feldman,Erica Meredith & Erik Stiller,"February 5, 2020",1422,3.74[315]
7,322,8,"""Family Tree""",Alec Smight,Bruce Zimmerman,"February 12, 2020",1423,3.94[316]
8,323,9,"""Face Off""",Sharat Raju,Christopher Barbour,"February 19, 2020",1424,5.46[317]
9,324,10,"""And in the End""",Glenn Kershaw,Erica Messer & Kirsten Vangsness,"February 19, 2020",1425,5.36[317]


In [9]:
#rename the column and make change permanent in the dataframe
wiki_html[15].rename(columns={
    'U.S. viewers (millions)':'US viewers (millions)'}, inplace=True)

In [10]:
#iterate through the tables we need and append them
#empty list for our resulting data
cm_wiki_data = []

#range from season 1 to 15 (indices 1, 15)
for i in range(1,16):
    cm_wiki_data.append(wiki_html[i])

In [11]:
cm_wiki_df = pd.concat(cm_wiki_data)
cm_wiki_df

,No. overall,No. in season,Title,Directed by,Written by,Original air date,Prod. code,US viewers (millions)
0,1,1,"""Extreme Aggressor""",Richard Shepard,Jeff Davis,"September 22, 2005",101,19.57[2]
1,2,2,"""Compulsion""",Charles Haid,Jeff Davis,"September 28, 2005",102,10.57[3]
2,3,3,"""Won't Get Fooled Again""",Kevin Bray,Aaron Zelman,"October 5, 2005",103,11.98[4]
3,4,4,"""Plain Sight""",Matt Earl Beesley,Edward Allen Bernero,"October 12, 2005",104,13.76[5]
4,5,5,"""Broken Mirror""",Guy Norman Bee,Judith McCreary,"October 19, 2005",105,12.79[6]
...,...,...,...,...,...,...,...,...
5,320,6,"""Date Night""",Marcus Stokes,Breen Frazier,"February 5, 2020",1420,4.35[315]
6,321,7,"""Rusty""",Rachel Feldman,Erica Meredith & Erik Stiller,"February 5, 2020",1422,3.74[315]
7,322,8,"""Family Tree""",Alec Smight,Bruce Zimmerman,"February 12, 2020",1423,3.94[316]
8,323,9,"""Face Off""",Sharat Raju,Christopher Barbour,"February 19, 2020",1424,5.46[317]


We now have all the relevant data from the wikipedia page compiled into a single dataset. 

#### 2.2: Scraping Data from IMDB

In [12]:
#empty lists of features that will compose dataframe
season_number_lst = []
episode_number_lst = []
episode_title_lst = []
episode_description_lst = []
imdb_rating_lst = []
imdb_votes_lst = []

In [13]:
#get the html documents for each season's page from imdb
#criminal minds has 15 seasons
for season in range(15):
    season_number = season + 1
    #print(f'--Extracting Data for Season {season_number}')
    imdb_url = f'https://www.imdb.com/title/tt0452046/episodes?season={season_number}' 
    
    #each season as its own page hence the 'season=' with our variable
    imdb_response = get(imdb_url)
    
    #response.status_code - 200 is connection established 

    season_html = BeautifulSoup(imdb_response.content)
    season_info = season_html.findAll('div', attrs={
        'class':'info'})
    
    #retrieve on the relevant data from each season's retrieve html docs
    for episode_number, episode in enumerate(season_info):
        episode_title = episode.strong.a.text
        #print(f'episode title: {episode_title}')
        
        episode_description = episode.find(attrs={
            'class':'item_description'}).text
        #print(f'summary: {episode_description}')
        
        imdb_rating = episode.find(attrs={
            'class':'ipl-rating-star__rating'}).text
        #print(f'episode name: {imdb_rating}')
        
        imdb_votes = episode.find(attrs={
            'class':'ipl-rating-star__total-votes'}).text
        #print(f'votes on imdb: {imdb_votes}')
        
        #print(f'\n')
        
        season_number_lst.append(season_number)
        episode_number_lst.append(episode_number + 1)
        episode_title_lst.append(episode_title)
        episode_description_lst.append(episode_description)
        imdb_rating_lst.append(imdb_rating)
        imdb_votes_lst.append(imdb_votes)

In [14]:
#create a dataframe using our outputs
cm_imdb_df = pd.DataFrame({
        'season_number':season_number_lst,
        'episode_number':episode_number_lst,
        'episode_title':episode_title_lst,
        'episode_description':episode_description_lst,
        'imdb_rating':imdb_rating_lst,
        'imdb_votes':imdb_votes_lst
})

In [15]:
cm_imdb_df

,season_number,episode_number,episode_title,episode_description,imdb_rating,imdb_votes
0,1,1,Extreme Aggressor,\nThe team travels to Seattle to find the capt...,7.8,"(2,969)"
1,1,2,Compulsion,\nThe team are called to an Arizona college wh...,7.6,"(2,486)"
2,1,3,Won't Get Fooled Again,\nA bomber in Palm Beach forces Gideon to conf...,7.6,"(2,460)"
3,1,4,Plain Sight,\nA serial rapist-killer active in San Diego t...,7.7,"(2,236)"
4,1,5,Broken Mirror,\nOne of a lawyer's identical twin daughters i...,7.9,"(2,319)"
...,...,...,...,...,...,...
318,15,6,Date Night,"\nAfter a father and daughter get kidnapped, a...",8.6,(841)
319,15,7,Rusty,\nWhen the BAU team travels to Denver to inves...,7.1,(550)
320,15,8,Family Tree,\nPrentiss and J.J. decide about their future ...,7.3,(524)
321,15,9,Face Off,\nIt has been a year since Rossi nearly died a...,8.2,(567)


### 3.0: Data Cleaning

- Use some string methods like .strip() and .replace() for remove punctuation marks from the values where none is needed.<br>
- Drop some column(s)<br>
- Convert string data type features into numeric features - for the features that need calculations
- Clean the date column

#### 3.1: Cleaning the Wikipedia Data

In [16]:
cm_wiki_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   No. overall            324 non-null    int64 
 1   No. in season          324 non-null    int64 
 2   Title                  324 non-null    object
 3   Directed by            324 non-null    object
 4   Written by             324 non-null    object
 5   Original air date      324 non-null    object
 6   Prod. code             324 non-null    int64 
 7   US viewers (millions)  324 non-null    object
dtypes: int64(3), object(5)
memory usage: 22.8+ KB


In [17]:
cm_wiki_df.drop(columns = 'Prod. code', inplace=True)

#remove quotation marks for the Titles
cm_wiki_df.Title = cm_wiki_df['Title'].str.strip('""')

#clean 'US viewers (millions)' column
#get the first 4 characters
cm_wiki_df['US viewers (millions)'] = [x[:5] for x in cm_wiki_df['US viewers (millions)']]
cm_wiki_df['US viewers (millions)'] = cm_wiki_df['US viewers (millions)'].str.strip('[')

cm_wiki_df['US viewers (millions)'] = pd.to_numeric(cm_wiki_df['US viewers (millions)'])

In [18]:
cm_wiki_df.dtypes

No. overall                int64
No. in season              int64
Title                     object
Directed by               object
Written by                object
Original air date         object
US viewers (millions)    float64
dtype: object

In [19]:
cm_wiki_df.columns

Index(['No. overall', 'No. in season', 'Title', 'Directed by', 'Written by',
       'Original air date', 'US viewers (millions)'],
      dtype='object')

In [20]:
cm_wiki_df.rename(columns={
    "Title": "episode_title",
    "No. overall":"episode_number_overall",
    "No. in season":"episode_number",
    "Directed by":"episode_director",
    "Written by":"episode_writer",
    "Original air date":"episode_airdate",
    "US viewers (millions)":"us_viewers_in_millions"
}, inplace=True)

In [21]:
#convert 'episode_airdate' into datetime
cm_wiki_df['episode_airdate'] = cm_wiki_df['episode_airdate'].str.replace(",","")

cm_wiki_df['episode_airdate'] = pd.to_datetime(cm_wiki_df['episode_airdate'])

#### 3.2: Cleaning the IMDB Data

In [22]:
#checking out data types
cm_imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   season_number        323 non-null    int64 
 1   episode_number       323 non-null    int64 
 2   episode_title        323 non-null    object
 3   episode_description  323 non-null    object
 4   imdb_rating          323 non-null    object
 5   imdb_votes           323 non-null    object
dtypes: int64(2), object(4)
memory usage: 15.3+ KB


In [23]:
#clean up the string features
cm_imdb_df.episode_description = cm_imdb_df['episode_description'].str.strip()

#convert data types 
cm_imdb_df['imdb_votes'] = cm_imdb_df['imdb_votes'].str.strip('()').str.replace(",", "").astype(int)

cm_imdb_df.imdb_rating = pd.to_numeric(cm_imdb_df.imdb_rating)

In [24]:
#cm_imdb_df

In [25]:
#cm_imdb_df.dtypes

our imdb dataframe have 323 episodes, while the wikipedia dataframe has 324 episodes. 

In [26]:
#check the number of episodes per season in the imdb df
cm_imdb_df.groupby(['season_number'])['episode_number'].count()

season_number
1     22
2     23
3     20
4     25
5     23
6     24
7     24
8     24
9     24
10    23
11    22
12    22
13    22
14    15
15    10
Name: episode_number, dtype: int64

In [27]:
#first table with seasons overview on wikipedia
wiki_html[0]

Season Episodes               Originally aired                    Rank  \
   Season Episodes Episodes.1         First aired         Last aired Rank   
0       1       22         22  September 22, 2005       May 10, 2006   27   
1       2       23         23  September 20, 2006       May 16, 2007   18   
2       3       20         20  September 26, 2007       May 21, 2008   18   
3       4       26         26  September 24, 2008       May 20, 2009   11   
4       5       23         23  September 23, 2009       May 26, 2010   14   
5       6       24         24  September 22, 2010       May 18, 2011   10   
6       7       24         24  September 21, 2011       May 16, 2012   13   
7       8       24         24  September 26, 2012       May 22, 2013   16   
8       9       24         24  September 25, 2013       May 14, 2014   13   
9      10       23         23     October 1, 2014        May 6, 2015    8   
10     11       22         22  September 30, 2015        May 4, 2016   13   
11     12       22         22  September 28, 2016       May 10, 2017   17   
12     13       22         22  September 27, 2017     April 18, 2018   25   
13     14       15         15     October 3, 2018   February 6, 2019    —   
14     15       10         10     January 8, 2020  February 19, 2020    —   

   Rating  
   Rating  
0     8.2  
1     8.8  
2     8.2  
3     9.4  
4     8.5  
5     8.7  
6     8.6  
7     8.0  
8     8.2  
9     9.0  
10    7.8  
11    6.9  
12    6.0  
13      —  
14      —

season 4 has 25 episodes in one df and 26 in another

In [28]:
#view season 4 from the wiki scraping
#wiki_html[4]

In [29]:
#view season 4 in the imdb df
cm_imdb_df[cm_imdb_df['season_number']==4]

,season_number,episode_number,episode_title,episode_description,imdb_rating,imdb_votes
65,4,1,Mayhem,"As the city is on high alert, the team begins ...",8.5,2847
66,4,2,The Angel Maker,On the anniversary of a serial killer's execut...,7.7,2031
67,4,3,Minimal Loss,"Reid and Prentiss, having gone undercover into...",8.2,2656
68,4,4,Paradise,A serial killer targeting couples stages their...,7.6,1918
69,4,5,Catching Out,"A man is jumping trains along Highway 99, brut...",7.3,1911
70,4,6,The Instincts,A child abduction case in Las Vegas causes Rei...,8.1,2593
71,4,7,Memoriam,"Driven by his nightmares, Reid stays behind in...",8.4,2695
72,4,8,Masterpiece,A serial killer turns himself in and challenge...,8.3,2275
73,4,9,52 Pickup,A greatly sadistic serial killer who disembowe...,7.9,2103
74,4,10,Brothers in Arms,"Morgan, an ex-cop, takes it personally when a ...",7.6,1805


As a fan of the show I remember that in the original airing, Episodes 25 & 26 of season was aired as one 2 hour long episode "To Hell...And Back". On wikipedia it is split as two separate episodes 25. To Hell & 26. And Back; while IMDB regards it as one.

In [30]:
#delete the row that contains the split part in the wikipedia df
cm_wiki_df = cm_wiki_df[~cm_wiki_df.episode_title.str.contains("And Back", na=False)]

#index of season 4 episode 25
#cm_wiki_df[cm_wiki_df.episode_title.str.contains("To Hell...") & (cm_wiki_df['episode_number'] == 90)].index

#change episode name of episode 5 in the imdb df, using .loc() with its row index and column index
cm_wiki_df.iloc[89, 2] = 'To Hell... And Back'

In [31]:
#confirm the change
cm_wiki_df.iloc[89]

episode_number_overall                     90
episode_number                             25
episode_title             To Hell... And Back
episode_director                 Charles Haid
episode_writer                    Chris Mundy
episode_airdate           2009-05-20 00:00:00
us_viewers_in_millions                  13.99
Name: 24, dtype: object

In [32]:
cm_wiki_df['episode_number_overall'] = range(1, 324)
cm_imdb_df['episode_number_overall'] = range(1, 324)

### 4.0: Combining Our Dataframes Into One

In [33]:
cm_imdb_df.head(2)

,season_number,episode_number,episode_title,episode_description,imdb_rating,imdb_votes,episode_number_overall
0,1,1,Extreme Aggressor,The team travels to Seattle to find the captor...,7.8,2969,1
1,1,2,Compulsion,The team are called to an Arizona college when...,7.6,2486,2


In [34]:
cm_wiki_df.head(2)

,episode_number_overall,episode_number,episode_title,episode_director,episode_writer,episode_airdate,us_viewers_in_millions
0,1,1,Extreme Aggressor,Richard Shepard,Jeff Davis,2005-09-22,19.57
1,2,2,Compulsion,Charles Haid,Jeff Davis,2005-09-28,10.57


In [35]:
cm_wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   episode_number_overall  323 non-null    int64         
 1   episode_number          323 non-null    int64         
 2   episode_title           323 non-null    object        
 3   episode_director        323 non-null    object        
 4   episode_writer          323 non-null    object        
 5   episode_airdate         323 non-null    datetime64[ns]
 6   us_viewers_in_millions  323 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 20.2+ KB


In [36]:
cm_imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_number           323 non-null    int64  
 1   episode_number          323 non-null    int64  
 2   episode_title           323 non-null    object 
 3   episode_description     323 non-null    object 
 4   imdb_rating             323 non-null    float64
 5   imdb_votes              323 non-null    int32  
 6   episode_number_overall  323 non-null    int64  
dtypes: float64(1), int32(1), int64(3), object(2)
memory usage: 16.5+ KB


both our dataframes have 323 rows with no null/NaN values

In [37]:
cm_df = pd.merge(cm_wiki_df, cm_imdb_df, on='episode_number_overall', how='left')

In [38]:
cm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 0 to 322
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   episode_number_overall  323 non-null    int64         
 1   episode_number_x        323 non-null    int64         
 2   episode_title_x         323 non-null    object        
 3   episode_director        323 non-null    object        
 4   episode_writer          323 non-null    object        
 5   episode_airdate         323 non-null    datetime64[ns]
 6   us_viewers_in_millions  323 non-null    float64       
 7   season_number           323 non-null    int64         
 8   episode_number_y        323 non-null    int64         
 9   episode_title_y         323 non-null    object        
 10  episode_description     323 non-null    object        
 11  imdb_rating             323 non-null    float64       
 12  imdb_votes              323 non-null    int32     

In [39]:
cm_df

,episode_number_overall,episode_number_x,episode_title_x,episode_director,episode_writer,episode_airdate,us_viewers_in_millions,season_number,episode_number_y,episode_title_y,episode_description,imdb_rating,imdb_votes
0,1,1,Extreme Aggressor,Richard Shepard,Jeff Davis,2005-09-22,19.57,1,1,Extreme Aggressor,The team travels to Seattle to find the captor...,7.8,2969
1,2,2,Compulsion,Charles Haid,Jeff Davis,2005-09-28,10.57,1,2,Compulsion,The team are called to an Arizona college when...,7.6,2486
2,3,3,Won't Get Fooled Again,Kevin Bray,Aaron Zelman,2005-10-05,11.98,1,3,Won't Get Fooled Again,A bomber in Palm Beach forces Gideon to confro...,7.6,2460
3,4,4,Plain Sight,Matt Earl Beesley,Edward Allen Bernero,2005-10-12,13.76,1,4,Plain Sight,A serial rapist-killer active in San Diego tar...,7.7,2236
4,5,5,Broken Mirror,Guy Norman Bee,Judith McCreary,2005-10-19,12.79,1,5,Broken Mirror,One of a lawyer's identical twin daughters is ...,7.9,2319
...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,319,6,Date Night,Marcus Stokes,Breen Frazier,2020-02-05,4.35,15,6,Date Night,"After a father and daughter get kidnapped, a f...",8.6,841
319,320,7,Rusty,Rachel Feldman,Erica Meredith & Erik Stiller,2020-02-05,3.74,15,7,Rusty,When the BAU team travels to Denver to investi...,7.1,550
320,321,8,Family Tree,Alec Smight,Bruce Zimmerman,2020-02-12,3.94,15,8,Family Tree,Prentiss and J.J. decide about their future as...,7.3,524
321,322,9,Face Off,Sharat Raju,Christopher Barbour,2020-02-19,5.46,15,9,Face Off,It has been a year since Rossi nearly died at ...,8.2,567


In [40]:
#rearrange the columns
cm_df = cm_df[['season_number','episode_number_overall','episode_number_x','episode_title_x','episode_description','episode_airdate','episode_director','episode_writer','imdb_rating','imdb_votes','us_viewers_in_millions']]
#cm_df
cm_df.rename(columns={"episode_number_x":"episode_number","episode_title_x":"episode_title"}, inplace=True)

In [41]:
cm_df.isnull().sum()

season_number             0
episode_number_overall    0
episode_number            0
episode_title             0
episode_description       0
episode_airdate           0
episode_director          0
episode_writer            0
imdb_rating               0
imdb_votes                0
us_viewers_in_millions    0
dtype: int64

In [42]:
cm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 0 to 322
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   season_number           323 non-null    int64         
 1   episode_number_overall  323 non-null    int64         
 2   episode_number          323 non-null    int64         
 3   episode_title           323 non-null    object        
 4   episode_description     323 non-null    object        
 5   episode_airdate         323 non-null    datetime64[ns]
 6   episode_director        323 non-null    object        
 7   episode_writer          323 non-null    object        
 8   imdb_rating             323 non-null    float64       
 9   imdb_votes              323 non-null    int32         
 10  us_viewers_in_millions  323 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(4)
memory usage: 29.0+ KB


### 5.0: Saving/Exporting Our Final Data

In [43]:
from pathlib import Path

In [44]:
#use pathlib to create a new folder
#Path.cwd() defines our currect working directory
output_path = Path.cwd() / 'scraping_output'
output_path.mkdir(exist_ok=True)

#export dataframe to csv into the 'scraping_output' folder
#exclude index from the columns
cm_df.to_csv("scraping_output/criminalminds-tv-scraped-data.csv", index=False)

In [45]:
%store cm_df

Stored 'cm_df' (DataFrame)
